<a href="https://colab.research.google.com/github/khroushan/embedding_layer_experiment/blob/main/markov_analysis_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### TODO:
- [ ] function to standardize a count sparse matrix
  
Later when we have transition matrix (TM) for the whole corpus and TM for a single data point to take the difference the sub matrix must renormalized again.

- [ ] import `imdb` data from keras (it is been already converted to integer indices). Concatenate all entries to large text (corpus)
- [ ] apply markov transition counter to both corpus and every entry separately.  

In [2]:
import numpy as np
from tensorflow import keras

In [3]:
some_text = """Zebras are primarily grazers and can subsist on lower-quality 
            vegetation. They are preyed on mainly by lions and typically flee 
            when threatened but also bite and kick. Zebra species differ in 
            social behaviour, with plains and mountain zebra living in stable 
            harems consisting of an adult male or stallion, several adult 
            females or mares, and their young or foals; while Grévy's zebra 
            live alone or in loosely associated herds. In harem-holding species, 
            adult females mate only with their harem stallion, while male 
            Grévy's zebras establish territories which attract females and the 
            species is promiscuous. Zebras communicate with various 
            vocalisations, body postures and facial expressions. Social 
            grooming strengthens social bonds in plains and mountain zebras.
            """

In [4]:
### method to extract the Markov transition matrix
def shift(in_tup, element):
    return in_tup[1:] + (element,)

def markov_dict_extractor(events: list, mk_len = 2):
  """ Count number of transition from state `w_i` to `w_i+1`. Calculate the 
  transition probabilities for available states.

  input:
  ------
  events: list, sequence of states
  mk_len: int, number of states gathered

  output:
  -------
  transition_dict: dict of tuples: count transition from one state to another.
  transition_norm: dict of tuples: transition rate (prob.)

  TODO:
  - the norm is not correct, the sum of prob. in each columns and rows
  must be one.
  - a function that convert the transition dictionary to a sparse matrix 
  representation with dictionary of indx and jndx to corresponding words.
  """
  events.append('EMPTY') # to count the last element as well
  transition_dic = {}
  pre = ()
  for event in events:
      if len(pre) < mk_len:
          pre +=  (event,)
      else:
          try:
              transition_dic[pre] += 1
          except: 
              transition_dic[pre] = 1
          pre = shift(pre, event)
          
          
  return transition_dic

def get_vocabs_dict(text: str, lowercase=True):
  """ Get vocaburaly of the text

    - convert it to lower case
  """
  vocaburaly = set(text.lower().split())
  # reserve 0 for unknown vocab in the text
  vocab_dict = {v:i for v,i in zip(vocaburaly, range(1, len(vocaburaly)+1))}
  return vocab_dict


In [5]:
transition_dict = markov_dict_extractor(some_text.lower().rstrip().split())

In [6]:
len(get_vocabs_dict(some_text))

78

In [7]:
sub_text = """ Zebras communicate with various 
            vocalisations, body postures and facial expressions
            """

In [8]:
sub_dict = markov_dict_extractor(some_text.lower().rstrip().split(), mk_len=2)
sub_dict_1 = markov_dict_extractor(some_text.lower().rstrip().split(), mk_len=1)

In [9]:
sub_dict.keys()

dict_keys([('zebras', 'are'), ('are', 'primarily'), ('primarily', 'grazers'), ('grazers', 'and'), ('and', 'can'), ('can', 'subsist'), ('subsist', 'on'), ('on', 'lower-quality'), ('lower-quality', 'vegetation.'), ('vegetation.', 'they'), ('they', 'are'), ('are', 'preyed'), ('preyed', 'on'), ('on', 'mainly'), ('mainly', 'by'), ('by', 'lions'), ('lions', 'and'), ('and', 'typically'), ('typically', 'flee'), ('flee', 'when'), ('when', 'threatened'), ('threatened', 'but'), ('but', 'also'), ('also', 'bite'), ('bite', 'and'), ('and', 'kick.'), ('kick.', 'zebra'), ('zebra', 'species'), ('species', 'differ'), ('differ', 'in'), ('in', 'social'), ('social', 'behaviour,'), ('behaviour,', 'with'), ('with', 'plains'), ('plains', 'and'), ('and', 'mountain'), ('mountain', 'zebra'), ('zebra', 'living'), ('living', 'in'), ('in', 'stable'), ('stable', 'harems'), ('harems', 'consisting'), ('consisting', 'of'), ('of', 'an'), ('an', 'adult'), ('adult', 'male'), ('male', 'or'), ('or', 'stallion,'), ('stallion

In [30]:
# sparse representation of transition matrix
def transition_mtx_to_sparse(trans_dict: dict, mk_len=2):
  """ Extract i-index, j-index and value transition 
  from transition dict 
  """
  if mk_len == 2:
    indx = [c[0] for c in trans_dict.keys()]
    jndx = [c[1] for c in trans_dict.keys()]
    cnt_vect = [v for v in trans_dict.values()] 
  elif mk_len == 1:
    indx = [c[0] for c in trans_dict.keys()]
    jndx = []
    cnt_vect = [v for v in trans_dict.values()]
  else:
    print("order of markov unit must be <= 2 !")

  return np.array(indx), np.array(jndx), np.array(cnt_vect)

In [31]:
indx, jndx, cnt_vect = transition_mtx_to_sparse(sub_dict, mk_len=2)
words, _, counts = transition_mtx_to_sparse(sub_dict_1, mk_len=1)
print(len(indx), len(jndx), len(cnt_vect))
word_counts = dict(zip(words, counts))

110 110 110


\begin{array}{cccccccccc}
      & and & in & is & most & nature & of & the & this & which\\
and   & 0   & 0  & 0  & 0    & 0      & 0  & 0   & 0    & 1\\
in    & 1   & 0  & 2  & 0    & 0      & 0  & 1   & 0    & 0\\
is    & 0   & 3  & 0  & 0    & 0      & 0  & 1   & 0    & 0\\
most  & 0   & 0  & 1  & 0    & 0      & 0  & 0   & 0    & 0\\
nature& 0   & 0  & 0  & 0    & 0      & 1  & 0   & 0    & 0\\
of    & 0   & 1  & 0  & 0    & 0      & 0  & 0   & 0    & 0\\
the   & 0   & 0  & 0  & 1    & 1      & 0  & 0   & 0    & 0\\
this  & 0   & 0  & 1  & 0    & 0      & 0  & 0   & 0    & 0\\
which & 0   & 0  & 0  & 0    & 0      & 0  & 0   & 0    & 0 
\end{array}

In [35]:
text = 'this is in the nature of in is the most is in is in and which'
dict_text = markov_dict_extractor(text.rstrip().split())
word_counts = markov_dict_extractor(text.rstrip().split(), 1)
indx, jndx, cnt_vect = transition_mtx_to_sparse(dict_text, mk_len=2)
## dict_text
# {('and', 'which'): 1,
#  ('in', 'and'): 1,
#  ('in', 'is'): 2,
#  ('in', 'the'): 1,
#  ('is', 'in'): 3,
#  ('is', 'the'): 1,
#  ('most', 'is'): 1,
#  ('nature', 'of'): 1,
#  ('of', 'in'): 1,
#  ('the', 'most'): 1,
#  ('the', 'nature'): 1,
#  ('this', 'is'): 1}
print(dict_text)
## word_counts
# {('and',): 1,
#  ('in',): 4,
#  ('is',): 4,
#  ('most',): 1,
#  ('nature',): 1,
#  ('of',): 1,
#  ('the',): 2,
#  ('this',): 1,
#  ('which',): 1}

{('this', 'is'): 1, ('is', 'in'): 3, ('in', 'the'): 1, ('the', 'nature'): 1, ('nature', 'of'): 1, ('of', 'in'): 1, ('in', 'is'): 2, ('is', 'the'): 1, ('the', 'most'): 1, ('most', 'is'): 1, ('in', 'and'): 1, ('and', 'which'): 1}


In [36]:
for i,j,k in zip(indx, jndx, cnt_vect):
  print(i,j,k)

this is 1
is in 3
in the 1
the nature 1
nature of 1
of in 1
in is 2
is the 1
the most 1
most is 1
in and 1
and which 1


In [41]:
np.unique(indx)

array(['and', 'in', 'is', 'most', 'nature', 'of', 'the', 'this'],
      dtype='<U6')

In [44]:
word_freq = {}
for w in np.unique(indx):
  word_freq[w] = cnt_vect[indx==w].sum()

In [45]:
word_freq

{'and': 1,
 'in': 4,
 'is': 4,
 'most': 1,
 'nature': 1,
 'of': 1,
 'the': 2,
 'this': 1}

In [48]:
def renormalize_mtx(indx, jndx, cnt_vect):
  """ Renormalize given sparse transition matrix
  """
  indx_unique_freq = {}
  for w in np.unique(indx):
    indx_unique_freq[w] = cnt_vect[indx==w].sum()

  cnt_renorm = np.zeros(cnt_vect.shape, float)
  for i,w in enumerate(indx):
    cnt_renorm[i] = cnt_vect[i]/indx_unique_freq[w]

  return indx, jndx, cnt_renorm

In [50]:
_, _ , vect_renomr = renormalize_mtx(indx, jndx, cnt_vect)

for v, w in zip(cnt_vect, vect_renomr):
  print(v,w)

1 1.0
3 0.75
1 0.25
1 0.5
1 1.0
1 1.0
2 0.5
1 0.25
1 0.5
1 1.0
1 0.25
1 1.0


In [19]:
def extract_markov_transition(events: list, mk_len = 2):
  """ Evaluate transition probability for a list of events
  """
  assert mk_len == 2
  # Evaluate transition counts
  dict_text = markov_dict_extractor(events, mk_len=2)
  word_counts = markov_dict_extractor(events, mk_len=1)
  
  trans_prob = {}
  for k in dict_text.keys():
    # Divide by total occurance of each keywords
    trans_prob[k] = dict_text[k]/word_counts[(k[0],)]

  return trans_prob   

In [20]:
extract_markov_transition(some_text.lower().rstrip().split())

{('adult', 'females'): 0.6666666666666666,
 ('adult', 'male'): 0.3333333333333333,
 ('alone', 'or'): 1.0,
 ('also', 'bite'): 1.0,
 ('an', 'adult'): 1.0,
 ('and', 'can'): 0.125,
 ('and', 'facial'): 0.125,
 ('and', 'kick.'): 0.125,
 ('and', 'mountain'): 0.25,
 ('and', 'the'): 0.125,
 ('and', 'their'): 0.125,
 ('and', 'typically'): 0.125,
 ('are', 'preyed'): 0.5,
 ('are', 'primarily'): 0.5,
 ('associated', 'herds.'): 1.0,
 ('attract', 'females'): 1.0,
 ('behaviour,', 'with'): 1.0,
 ('bite', 'and'): 1.0,
 ('body', 'postures'): 1.0,
 ('bonds', 'in'): 1.0,
 ('but', 'also'): 1.0,
 ('by', 'lions'): 1.0,
 ('can', 'subsist'): 1.0,
 ('communicate', 'with'): 1.0,
 ('consisting', 'of'): 1.0,
 ('differ', 'in'): 1.0,
 ('establish', 'territories'): 1.0,
 ('expressions.', 'social'): 1.0,
 ('facial', 'expressions.'): 1.0,
 ('females', 'and'): 0.3333333333333333,
 ('females', 'mate'): 0.3333333333333333,
 ('females', 'or'): 0.3333333333333333,
 ('flee', 'when'): 1.0,
 ('foals;', 'while'): 1.0,
 ('grazers